In [4]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

# Obtener información del compuesto K3Fe(CN)6 (Potassium ferrocyanide)
compound = pcp.get_compounds('K3Fe(CN)6', 'name')[0]
smiles = compound.isomeric_smiles

# Crear una molécula RDKit desde SMILES
mol = Chem.MolFromSmiles(smiles)
if mol is None:
    raise ValueError("No se pudo crear la molécula desde SMILES")

# Calcular la masa molar usando RDKit
mol_weight = rdMolDescriptors.CalcExactMolWt(mol)
print(f"Masa molar de K3Fe(CN)6: {mol_weight:.2f} g/mol")

# Parámetros del problema
ppm_K3FeCN6 = 63.3  # ppm
density_solution = 1.00  # g/mL (asumida)

# Convertir ppm a mg/L (asumiendo densidad 1.00 g/mL)
mg_per_L = ppm_K3FeCN6  # 63.3 mg/L

# Convertir mg/L a g/L
g_per_L = mg_per_L / 1000  # 0.0633 g/L

# Calcular moles de K3Fe(CN)6 por litro
moles_K3FeCN6_per_L = g_per_L / mol_weight  # mol/L

# Cada mol de K3Fe(CN)6 produce 3 moles de K+
moles_K_per_L = moles_K3FeCN6_per_L * 3  # mol/L

print(f"Concentración molar de K3Fe(CN)6: {moles_K3FeCN6_per_L:.4e} M")
print(f"Concentración molar de K+: {moles_K_per_L:.4e} M")

Masa molar de K3Fe(CN)6: 328.84 g/mol
Concentración molar de K3Fe(CN)6: 1.9249e-04 M
Concentración molar de K+: 5.7748e-04 M


In [6]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import re

def parse_chemical_formula(formula):
    """
    Analiza una fórmula química y devuelve los elementos con sus cantidades
    Ej: K3Fe(CN)6 -> {'K': 3, 'Fe': 1, 'C': 6, 'N': 6}
    """
    # Expandir grupos entre paréntesis
    while '(' in formula:
        # Buscar el último grupo de paréntesis
        match = re.search(r'\(([^()]+)\)(\d*)', formula)
        if not match:
            break
        
        group = match.group(1)
        multiplier = int(match.group(2)) if match.group(2) else 1
        
        # Expandir el grupo
        expanded = ''
        for element_match in re.finditer(r'([A-Z][a-z]?)(\d*)', group):
            element = element_match.group(1)
            count = int(element_match.group(2)) if element_match.group(2) else 1
            expanded += element + str(count * multiplier)
        
        # Reemplazar en la fórmula
        formula = formula[:match.start()] + expanded + formula[match.end():]
    
    # Contar elementos
    elements = {}
    for match in re.finditer(r'([A-Z][a-z]?)(\d*)', formula):
        element = match.group(1)
        count = int(match.group(2)) if match.group(2) else 1
        elements[element] = elements.get(element, 0) + count
    
    return elements

def find_element_multiplier(compound_formula, target_element):
    """
    Encuentra cuántos átomos de un elemento hay en un compuesto
    """
    try:
        elements = parse_chemical_formula(compound_formula)
        
        # Buscar el elemento objetivo (sin carga)
        target_base = re.sub(r'[+-]\d*$', '', target_element)  # Remover carga (ej: K+ -> K)
        
        if target_base in elements:
            return elements[target_base]
        else:
            print(f"Advertencia: No se encontró {target_base} en {compound_formula}")
            print(f"Elementos encontrados: {list(elements.keys())}")
            return None
            
    except Exception as e:
        print(f"Error al analizar la fórmula: {e}")
        return None

def calculate_molar_concentration():
    """Calculadora que calcula automáticamente la estequiometría"""
    
    print("="*60)
    print("CALCULADORA DE CONCENTRACIÓN MOLAR")
    print("="*60)
    
    # Pedir compuesto en solución
    solution_compound = input("\n1. Ingrese el compuesto en solución (ej: K3Fe(CN)6): ").strip()
    
    # Pedir concentración en ppm
    while True:
        try:
            ppm = float(input("2. Ingrese la concentración en ppm: "))
            break
        except ValueError:
            print("   Error: Ingrese un número válido")
    
    # Pedir componente específico a calcular
    target_component = input("3. Ingrese el elemento/ion que desea calcular (ej: K, K+, Fe, etc.): ").strip()
    
    print("\nProcesando...")
    
    try:
        # Calcular automáticamente el multiplicador estequiométrico
        multiplier = find_element_multiplier(solution_compound, target_component)
        
        if multiplier is None:
            print("No se pudo determinar la estequiometría automáticamente.")
            while True:
                try:
                    multiplier = int(input(f"Ingrese manualmente cuántos moles de {target_component} produce 1 mol de {solution_compound}: "))
                    break
                except ValueError:
                    print("   Error: Ingrese un número entero")
        else:
            print(f"Relación estequiométrica detectada: 1 mol {solution_compound} → {multiplier} mol {target_component}")
        
        # Obtener información del compuesto desde PubChem
        compound = pcp.get_compounds(solution_compound, 'name')[0]
        smiles = compound.isomeric_smiles
        
        # Crear molécula RDKit y calcular masa molar
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("No se pudo procesar el compuesto")
        
        mol_weight = rdMolDescriptors.CalcExactMolWt(mol)
        
        # Conversiones
        mg_per_L = ppm  # ppm = mg/L (asumiendo densidad ≈ 1.00 g/mL)
        g_per_L = mg_per_L / 1000
        moles_compound_per_L = g_per_L / mol_weight
        moles_target_per_L = moles_compound_per_L * multiplier
        
        # Mostrar resultados
        print(f"\n{'='*50}")
        print(f"RESULTADOS")
        print(f"{'='*50}")
        print(f"Compuesto en solución: {solution_compound}")
        print(f"Componente calculado: {target_component}")
        print(f"Masa molar de {solution_compound}: {mol_weight:.2f} g/mol")
        print(f"Concentración en ppm: {ppm:.2f}")
        print(f"Concentración molar de {solution_compound}: {moles_compound_per_L:.4e} M")
        print(f"Concentración molar de {target_component}: {moles_target_per_L:.4e} M")
        print(f"Relación estequiométrica: 1 mol {solution_compound} → {multiplier} mol {target_component}")
        
        # Mostrar composición del compuesto
        elements = parse_chemical_formula(solution_compound)
        print(f"\nComposición del compuesto:")
        for element, count in elements.items():
            print(f"  • {element}: {count} átomos")
        
        return {
            'solution_compound': solution_compound,
            'target_component': target_component,
            'molecular_weight': mol_weight,
            'ppm': ppm,
            'compound_molarity': moles_compound_per_L,
            'target_molarity': moles_target_per_L,
            'multiplier': multiplier,
            'composition': elements
        }
        
    except Exception as e:
        print(f"\nError: No se pudo procesar el compuesto '{solution_compound}'")
        print(f"Detalles del error: {e}")
        print("Verifique que el nombre del compuesto sea correcto")
        return None

def main():
    """Función principal"""
    while True:
        try:
            # Realizar cálculo
            results = calculate_molar_concentration()
            
            if results:
                # Preguntar si desea hacer otro cálculo
                print("\n" + "="*50)
                another = input("¿Desea realizar otro cálculo? (s/n): ").strip().lower()
                if another not in ['s', 'si', 'sí', 'y', 'yes']:
                    break
            else:
                # Si hubo error, preguntar si quiere intentar de nuevo
                retry = input("¿Desea intentar con otro compuesto? (s/n): ").strip().lower()
                if retry not in ['s', 'si', 'sí', 'y', 'yes']:
                    break
                    
        except KeyboardInterrupt:
            print("\n\nPrograma interrumpido.")
            break
        except Exception as e:
            print(f"\nError inesperado: {e}")
            break
    
    print("\n¡Gracias por usar la calculadora!")

# Ejecutar programa
if __name__ == "__main__":
    main()

CALCULADORA DE CONCENTRACIÓN MOLAR

Procesando...
Relación estequiométrica detectada: 1 mol K3Fe(CN)6 → 3 mol K+

RESULTADOS
Compuesto en solución: K3Fe(CN)6
Componente calculado: K+
Masa molar de K3Fe(CN)6: 328.84 g/mol
Concentración en ppm: 63.30
Concentración molar de K3Fe(CN)6: 1.9249e-04 M
Concentración molar de K+: 5.7748e-04 M
Relación estequiométrica: 1 mol K3Fe(CN)6 → 3 mol K+

Composición del compuesto:
  • K: 3 átomos
  • Fe: 1 átomos
  • C: 6 átomos
  • N: 6 átomos


¡Gracias por usar la calculadora!


Instalación requerida:
pip install pubchempy rdkit matplotlib python-docx pillow
conda install -c conda-forge rdkit

CALCULADORA DE CONCENTRACIÓN MOLAR

Procesando...
Relación estequiométrica detectada: 1 mol K3Fe(CN)6 → 3 mol K+

RESULTADOS
Compuesto en solución: K3Fe(CN)6
Componente calculado: K+
Masa molar de K3Fe(CN)6: 328.84 g/mol
Concentración en ppm: 63.30
Concentración molar de K3Fe(CN)6: 1.9249e-04 M
Concentración molar de K+: 5.7748e-04 M
Relación estequiométrica: 1 mol K3Fe(CN)6 → 3 mol K+

Composición del compuesto:
  • K: 3 átomos
  • Fe: 1 átomos
  • C: 6 átomos
  • N: 6 átomos

Generando visualizaciones moleculares...
✓ Estructura 2D guardada: estructura_2d_20250528_215020.png


[21:50:20] UFFTYPER: Unrecognized atom type: Fe2+2 (0)


✓ Estructura 3D guardada: estructura_3d_20250528_215020.png

Generando informe técnico...
✓ Informe técnico guardado: Informe_Concentracion_Molar_20250528_215020.docx


¡Gracias por usar la calculadora!


In [2]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors, Draw, AllChem
from rdkit.Chem.Draw import rdMolDraw2D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import RGBColor
import io
import base64
from PIL import Image
import os
from datetime import datetime
import re


def parse_chemical_formula(formula):
    """
    Analiza una fórmula química y devuelve los elementos con sus cantidades
    Ej: K3Fe(CN)6 -> {'K': 3, 'Fe': 1, 'C': 6, 'N': 6}
    """
    # Expandir grupos entre paréntesis
    while '(' in formula:
        # Buscar el último grupo de paréntesis
        match = re.search(r'\(([^()]+)\)(\d*)', formula)
        if not match:
            break
       
        group = match.group(1)
        multiplier = int(match.group(2)) if match.group(2) else 1
       
        # Expandir el grupo
        expanded = ''
        for element_match in re.finditer(r'([A-Z][a-z]?)(\d*)', group):
            element = element_match.group(1)
            count = int(element_match.group(2)) if element_match.group(2) else 1
            expanded += element + str(count * multiplier)
       
        # Reemplazar en la fórmula
        formula = formula[:match.start()] + expanded + formula[match.end():]
   
    # Contar elementos
    elements = {}
    for match in re.finditer(r'([A-Z][a-z]?)(\d*)', formula):
        element = match.group(1)
        count = int(match.group(2)) if match.group(2) else 1
        elements[element] = elements.get(element, 0) + count
   
    return elements


def find_element_multiplier(compound_formula, target_element):
    """
    Encuentra cuántos átomos de un elemento hay en un compuesto
    """
    try:
        elements = parse_chemical_formula(compound_formula)
       
        # Buscar el elemento objetivo (sin carga)
        target_base = re.sub(r'[+-]\d*$', '', target_element)  # Remover carga (ej: K+ -> K)
       
        if target_base in elements:
            return elements[target_base]
        else:
            print(f"Advertencia: No se encontró {target_base} en {compound_formula}")
            print(f"Elementos encontrados: {list(elements.keys())}")
            return None
           
    except Exception as e:
        print(f"Error al analizar la fórmula: {e}")
        return None


def generate_2d_structure(smiles, compound_name, save_path=None):
    """
    Genera una imagen 2D de la estructura molecular
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
       
        # Generar coordenadas 2D
        AllChem.Compute2DCoords(mol)
       
        # Crear imagen 2D
        drawer = rdMolDraw2D.MolDraw2DCairo(800, 600)
        drawer.DrawMolecule(mol)
        drawer.FinishDrawing()
       
        # Guardar imagen
        if save_path:
            with open(save_path, 'wb') as f:
                f.write(drawer.GetDrawingText())
       
        return drawer.GetDrawingText()
       
    except Exception as e:
        print(f"Error generando estructura 2D: {e}")
        return None


def generate_3d_structure(smiles, compound_name, save_path=None):
    """
    Genera una visualización 3D básica de la molécula
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
       
        # Generar conformador 3D
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol)
       
        # Obtener coordenadas 3D
        conf = mol.GetConformer()
        coords = []
        elements = []
       
        for atom in mol.GetAtoms():
            pos = conf.GetAtomPosition(atom.GetIdx())
            coords.append([pos.x, pos.y, pos.z])
            elements.append(atom.GetSymbol())
       
        coords = np.array(coords)
       
        # Crear gráfico 3D
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
       
        # Colores para diferentes elementos
        color_map = {
            'C': 'black', 'H': 'white', 'O': 'red', 'N': 'blue',
            'S': 'yellow', 'P': 'orange', 'F': 'green', 'Cl': 'green',
            'Br': 'brown', 'I': 'purple', 'K': 'violet', 'Fe': 'orange'
        }
       
        # Plotear átomos
        for i, (coord, element) in enumerate(zip(coords, elements)):
            color = color_map.get(element, 'gray')
            size = 100 if element != 'H' else 50
            ax.scatter(coord[0], coord[1], coord[2],
                      c=color, s=size, alpha=0.8, edgecolors='black')
           
            # Añadir etiquetas
            if element != 'H':  # No mostrar hidrógenos para claridad
                ax.text(coord[0], coord[1], coord[2], f'  {element}', fontsize=8)
       
        # Dibujar enlaces
        for bond in mol.GetBonds():
            atom1_idx = bond.GetBeginAtomIdx()
            atom2_idx = bond.GetEndAtomIdx()
           
            # Solo mostrar enlaces que no involucren hidrógenos
            if elements[atom1_idx] != 'H' and elements[atom2_idx] != 'H':
                coord1 = coords[atom1_idx]
                coord2 = coords[atom2_idx]
                ax.plot([coord1[0], coord2[0]],
                       [coord1[1], coord2[1]],
                       [coord1[2], coord2[2]], 'k-', alpha=0.6)
       
        ax.set_title(f'Estructura 3D: {compound_name}', fontsize=14, fontweight='bold')
        ax.set_xlabel('X (Å)')
        ax.set_ylabel('Y (Å)')
        ax.set_zlabel('Z (Å)')
       
        # Mejorar la visualización
        ax.grid(True, alpha=0.3)
       
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()
       
        return fig
       
    except Exception as e:
        print(f"Error generando estructura 3D: {e}")
        return None


def create_technical_report(results, images_info=None):
    """
    Crea un informe técnico en formato DOCX
    """
    try:
        # Crear documento
        doc = Document()
       
        # Configurar estilos
        styles = doc.styles
       
        # Título principal
        title = doc.add_heading('INFORME TÉCNICO DE ANÁLISIS DE CONCENTRACIÓN MOLAR', 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
       
        # Información general
        doc.add_heading('1. INFORMACIÓN GENERAL', level=1)
       
        info_table = doc.add_table(rows=5, cols=2)
        info_table.style = 'Table Grid'
       
        info_data = [
            ['Fecha de análisis:', datetime.now().strftime('%d/%m/%Y %H:%M:%S')],
            ['Compuesto analizado:', results['solution_compound']],
            ['Componente objetivo:', results['target_component']],
            ['Concentración inicial:', f"{results['ppm']:.2f} ppm"],
            ['Masa molar:', f"{results['molecular_weight']:.2f} g/mol"]
        ]
       
        for i, (key, value) in enumerate(info_data):
            info_table.cell(i, 0).text = key
            info_table.cell(i, 1).text = str(value)
       
        # Resultados del cálculo
        doc.add_heading('2. RESULTADOS DEL CÁLCULO', level=1)
       
        results_para = doc.add_paragraph()
        results_para.add_run(f"Concentración molar del compuesto ({results['solution_compound']}): ").bold = True
        results_para.add_run(f"{results['compound_molarity']:.4e} M\n")
       
        results_para.add_run(f"Concentración molar del componente ({results['target_component']}): ").bold = True
        results_para.add_run(f"{results['target_molarity']:.4e} M\n")
       
        results_para.add_run(f"Factor estequiométrico: ").bold = True
        results_para.add_run(f"1 mol {results['solution_compound']} → {results['multiplier']} mol {results['target_component']}")
       
        # Composición química
        doc.add_heading('3. COMPOSICIÓN QUÍMICA', level=1)
       
        comp_para = doc.add_paragraph(f"El compuesto {results['solution_compound']} está compuesto por:")
        for element, count in results['composition'].items():
            comp_para.add_run(f"\n• {element}: {count} átomos")
       
        # Cálculos detallados
        doc.add_heading('4. CÁLCULOS DETALLADOS', level=1)
       
        calc_para = doc.add_paragraph()
        calc_para.add_run("Paso 1: Conversión de ppm a g/L\n").bold = True
        calc_para.add_run(f"Concentración = {results['ppm']:.2f} ppm = {results['ppm']:.2f} mg/L = {results['ppm']/1000:.4f} g/L\n\n")
       
        calc_para.add_run("Paso 2: Cálculo de molaridad del compuesto\n").bold = True
        calc_para.add_run(f"Molaridad = (Concentración en g/L) / (Masa molar)\n")
        calc_para.add_run(f"Molaridad = {results['ppm']/1000:.4f} g/L / {results['molecular_weight']:.2f} g/mol = {results['compound_molarity']:.4e} M\n\n")
       
        calc_para.add_run("Paso 3: Cálculo de molaridad del componente objetivo\n").bold = True
        calc_para.add_run(f"Molaridad del {results['target_component']} = Molaridad del compuesto × Factor estequiométrico\n")
        calc_para.add_run(f"Molaridad del {results['target_component']} = {results['compound_molarity']:.4e} M × {results['multiplier']} = {results['target_molarity']:.4e} M")
       
        # Añadir imágenes si están disponibles
        if images_info:
            doc.add_heading('5. ESTRUCTURA MOLECULAR', level=1)
           
            # Imagen 2D
            if images_info.get('image_2d_path') and os.path.exists(images_info['image_2d_path']):
                doc.add_heading('5.1 Estructura 2D', level=2)
                doc.add_picture(images_info['image_2d_path'], width=Inches(4))
                doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
           
            # Imagen 3D
            if images_info.get('image_3d_path') and os.path.exists(images_info['image_3d_path']):
                doc.add_heading('5.2 Estructura 3D', level=2)
                doc.add_picture(images_info['image_3d_path'], width=Inches(4))
                doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
       
        # Conclusiones
        doc.add_heading('6. CONCLUSIONES', level=1)
       
        conclusion_para = doc.add_paragraph()
        conclusion_para.add_run("• ").bold = True
        conclusion_para.add_run(f"El compuesto {results['solution_compound']} a una concentración de {results['ppm']:.2f} ppm ")
        conclusion_para.add_run(f"equivale a una molaridad de {results['compound_molarity']:.4e} M.\n")
       
        conclusion_para.add_run("• ").bold = True
        conclusion_para.add_run(f"La concentración molar del componente {results['target_component']} ")
        conclusion_para.add_run(f"es de {results['target_molarity']:.4e} M.\n")
       
        conclusion_para.add_run("• ").bold = True
        conclusion_para.add_run(f"La relación estequiométrica es 1:{results['multiplier']} ")
        conclusion_para.add_run(f"({results['solution_compound']}:{results['target_component']}).")
       
        # Guardar documento
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"Informe_Concentracion_Molar_{timestamp}.docx"
        doc.save(filename)
       
        return filename
       
    except Exception as e:
        print(f"Error creando informe técnico: {e}")
        return None


def calculate_molar_concentration():
    """Calculadora que calcula automáticamente la estequiometría"""
   
    print("="*60)
    print("CALCULADORA DE CONCENTRACIÓN MOLAR")
    print("="*60)
   
    # Pedir compuesto en solución
    solution_compound = input("\n1. Ingrese el compuesto en solución (ej: K3Fe(CN)6): ").strip()
   
    # Pedir concentración en ppm
    while True:
        try:
            ppm = float(input("2. Ingrese la concentración en ppm: "))
            break
        except ValueError:
            print("   Error: Ingrese un número válido")
   
    # Pedir componente específico a calcular
    target_component = input("3. Ingrese el elemento/ion que desea calcular (ej: K, K+, Fe, etc.): ").strip()
   
    print("\nProcesando...")
   
    try:
        # Calcular automáticamente el multiplicador estequiométrico
        multiplier = find_element_multiplier(solution_compound, target_component)
       
        if multiplier is None:
            print("No se pudo determinar la estequiometría automáticamente.")
            while True:
                try:
                    multiplier = int(input(f"Ingrese manualmente cuántos moles de {target_component} produce 1 mol de {solution_compound}: "))
                    break
                except ValueError:
                    print("   Error: Ingrese un número entero")
        else:
            print(f"Relación estequiométrica detectada: 1 mol {solution_compound} → {multiplier} mol {target_component}")
       
        # Obtener información del compuesto desde PubChem
        compound = pcp.get_compounds(solution_compound, 'name')[0]
        smiles = compound.isomeric_smiles
       
        # Crear molécula RDKit y calcular masa molar
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("No se pudo procesar el compuesto")
       
        mol_weight = rdMolDescriptors.CalcExactMolWt(mol)
       ###################AQUI ES DONDE SE REALIZAN LOS CALCULOS DEL EJERICIO##################
        # Conversiones
        mg_per_L = ppm  # ppm = mg/L (asumiendo densidad ≈ 1.00 g/mL)
        g_per_L = mg_per_L / 1000
        moles_compound_per_L = g_per_L / mol_weight
        moles_target_per_L = moles_compound_per_L * multiplier
       
        # Mostrar resultados
        print(f"\n{'='*50}")
        print(f"RESULTADOS")
        print(f"{'='*50}")
        print(f"Compuesto en solución: {solution_compound}")
        print(f"Componente calculado: {target_component}")
        print(f"Masa molar de {solution_compound}: {mol_weight:.2f} g/mol")
        print(f"Concentración en ppm: {ppm:.2f}")
        print(f"Concentración molar de {solution_compound}: {moles_compound_per_L:.4e} M")
        print(f"Concentración molar de {target_component}: {moles_target_per_L:.4e} M")
        print(f"Relación estequiométrica: 1 mol {solution_compound} → {multiplier} mol {target_component}")
       
        # Mostrar composición del compuesto
        elements = parse_chemical_formula(solution_compound)
        print(f"\nComposición del compuesto:")
        for element, count in elements.items():
            print(f"  • {element}: {count} átomos")
       
        # Generar visualizaciones
        print("\nGenerando visualizaciones moleculares...")
       
        images_info = {}
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
       
        # Generar estructura 2D
        image_2d_path = f"estructura_2d_{timestamp}.png"
        structure_2d = generate_2d_structure(smiles, solution_compound, image_2d_path)
        if structure_2d:
            images_info['image_2d_path'] = image_2d_path
            print(f"✓ Estructura 2D guardada: {image_2d_path}")
       
        # Generar estructura 3D
        image_3d_path = f"estructura_3d_{timestamp}.png"
        structure_3d = generate_3d_structure(smiles, solution_compound, image_3d_path)
        if structure_3d:
            images_info['image_3d_path'] = image_3d_path
            print(f"✓ Estructura 3D guardada: {image_3d_path}")
       
        # Crear informe técnico
        print("\nGenerando informe técnico...")
       
        results_dict = {
            'solution_compound': solution_compound,
            'target_component': target_component,
            'molecular_weight': mol_weight,
            'ppm': ppm,
            'compound_molarity': moles_compound_per_L,
            'target_molarity': moles_target_per_L,
            'multiplier': multiplier,
            'composition': elements
        }
       
        report_filename = create_technical_report(results_dict, images_info)
        if report_filename:
            print(f"✓ Informe técnico guardado: {report_filename}")
       
        return results_dict
       
    except Exception as e:
        print(f"\nError: No se pudo procesar el compuesto '{solution_compound}'")
        print(f"Detalles del error: {e}")
        print("Verifique que el nombre del compuesto sea correcto")
        return None


def main():
    """Función principal"""
    print("Instalación requerida:")
    print("pip install pubchempy rdkit matplotlib python-docx pillow")
    print("conda install -c conda-forge rdkit")
    print("\n" + "="*60)
   
    while True:
        try:
            # Realizar cálculo
            results = calculate_molar_concentration()
           
            if results:
                # Preguntar si desea hacer otro cálculo
                print("\n" + "="*50)
                another = input("¿Desea realizar otro cálculo? (s/n): ").strip().lower()
                if another not in ['s', 'si', 'sí', 'y', 'yes']:
                    break
            else:
                # Si hubo error, preguntar si quiere intentar de nuevo
                retry = input("¿Desea intentar con otro compuesto? (s/n): ").strip().lower()
                if retry not in ['s', 'si', 'sí', 'y', 'yes']:
                    break
                   
        except KeyboardInterrupt:
            print("\n\nPrograma interrumpido.")
            break
        except Exception as e:
            print(f"\nError inesperado: {e}")
            break
   
    print("\n¡Gracias por usar la calculadora!")


# Ejecutar programa
if __name__ == "__main__":
    main()



Instalación requerida:
pip install pubchempy rdkit matplotlib python-docx pillow
conda install -c conda-forge rdkit

CALCULADORA DE CONCENTRACIÓN MOLAR

Procesando...
Relación estequiométrica detectada: 1 mol K3Fe(CN)6 → 3 mol K+

RESULTADOS
Compuesto en solución: K3Fe(CN)6
Componente calculado: K+
Masa molar de K3Fe(CN)6: 328.84 g/mol
Concentración en ppm: 63.30
Concentración molar de K3Fe(CN)6: 1.9249e-04 M
Concentración molar de K+: 5.7748e-04 M
Relación estequiométrica: 1 mol K3Fe(CN)6 → 3 mol K+

Composición del compuesto:
  • K: 3 átomos
  • Fe: 1 átomos
  • C: 6 átomos
  • N: 6 átomos

Generando visualizaciones moleculares...
✓ Estructura 2D guardada: estructura_2d_20250529_193909.png


[19:39:09] UFFTYPER: Unrecognized atom type: Fe2+2 (0)


✓ Estructura 3D guardada: estructura_3d_20250529_193909.png

Generando informe técnico...
✓ Informe técnico guardado: Informe_Concentracion_Molar_20250529_193909.docx


¡Gracias por usar la calculadora!
